In [1]:
# Add project root to sys.path
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import os
import uuid
import json
from dotenv import load_dotenv
from supabase import create_client, Client
# Assuming this is the correct alias for google.generativeai.types
# Also ensure types.GenerateContentConfig is available via this import
from google.genai import types
import traceback # Import traceback for better error logging

# Ensure you are using the correct import for Gemini's client library
# If google.genai.types is not found, it's likely google.generativeai.types
# try:
#     from google.genai import types
# except ImportError:
#     from google.generativeai import types


# Assuming these imports are correct based on your project structure
from src.llm.OpenAIClient import OpenAIClient
from src.llm.GeminiClient import GeminiClient
from src.storage.SupabaseService import SupabaseService
from src.enums import FinancialDocSpecificType

# --- HELPER FUNCTION: Serialize conversation history for printing ---
# This fixes the AttributeError when trying to print Content objects directly
def serialize_conversation_history(history: list) -> list:
    """
    Manually serializes conversation history (list of Content objects)
    into a list of dictionaries for printing.
    Handles basic text, function_call, and function_response parts.
    """
    serializable_history = []
    for content_item in history:
        # Check if content_item is already a dict (e.g., from a previous manual step)
        if isinstance(content_item, dict):
            serializable_history.append(content_item)
            continue

        # Assume it's a types.Content object or similar structure
        item_dict = {"role": content_item.role, "parts": []}
        # Ensure parts exist and are not None before iterating
        if hasattr(content_item, 'parts') and content_item.parts is not None:
            for part_item in content_item.parts:
                part_dict = {}
                # Check for text part
                if hasattr(part_item, 'text') and part_item.text is not None:
                    part_dict['text'] = part_item.text
                # Check for function_call part
                if hasattr(part_item, 'function_call') and part_item.function_call:
                    part_dict['function_call'] = {
                        "name": part_item.function_call.name,
                        # Convert Pydantic object to dict, handle empty args
                        "args": dict(part_item.function_call.args) if hasattr(part_item.function_call.args, 'items') else {}
                    }
                # Check for function_response part
                if hasattr(part_item, 'function_response') and part_item.function_response:
                    # The 'response' field in FunctionResponsePart itself contains the data
                    response_data = part_item.function_response.response
                    # Assuming response_data is the dict loaded from JSON from your tool
                    part_dict['function_response'] = {
                        "name": part_item.function_response.name,
                        "response": response_data
                    }
                # Check for other part types if needed (e.g., inline_data, file_data)
                # Example for inline_data (like images):
                # if hasattr(part_item, 'inline_data') and part_item.inline_data:
                #     part_dict['inline_data'] = {
                #         "mime_type": part_item.inline_data.mime_type,
                #         "data": f"...data_bytes_or_base64..." # Represent appropriately
                #     }

                if part_dict: # Only add if part_dict is not empty
                    item_dict["parts"].append(part_dict)
        serializable_history.append(item_dict)
    return serializable_history


# --- HELPER FUNCTION: Format retrieved chunks for the LLM ---
# This structures the retrieved data clearly for the LLM to use for answering and citing
def format_chunks_for_llm(retrieved_chunks_json: str) -> str:
    """
    Formats the JSON string of retrieved chunks into a readable text block
    for the LLM, including necessary details for citation.
    """
    try:
        chunks = json.loads(retrieved_chunks_json)
        # Handle empty or error responses from retrieve_financial_chunks
        if not chunks:
            return "No specific information snippets were found to answer the question.\n"
        if isinstance(chunks, dict) and "error" in chunks:
             return f"Could not retrieve information snippets: {chunks['error']}\n"


        formatted_text = "Okay, I have retrieved the following information snippets for you:\n\n"
        for i, chunk_data in enumerate(chunks):
            snippet_num = i + 1
            # Ensure these keys match what your `retrieve_financial_chunks` actually returns from the RPC
            # Added more robust checks and fallbacks here
            doc_name = chunk_data.get("document_filename")
            if doc_name is None:
                 doc_name = chunk_data.get("filename", "Unknown Document") # Fallback to 'filename' key if RPC uses that
                 print(f"  Warning: 'document_filename' missing from chunk_data, used fallback 'filename': {doc_name}")

            sec_id = chunk_data.get("section_id")
            if sec_id is None:
                 sec_id = chunk_data.get("id", "unknown_section_missing_from_rpc") # Fallback to chunk 'id'
                 print(f"  Warning: 'section_id' missing from chunk_data, used fallback 'id': {sec_id}")

            sec_heading = chunk_data.get("section_heading", "N/A") # Make sure this is returned by RPC
            chunk_text_content = chunk_data.get("chunk_text", "No content.")

            formatted_text += f"[Snippet {snippet_num}]\n"
            formatted_text += f"Source Document: {doc_name}\n"
            formatted_text += f"Source Section ID: {sec_id}\n"
            formatted_text += f"Source Section Heading: {sec_heading}\n" # Good for LLM context
            formatted_text += f"Content:\n{chunk_text_content}\n\n"

        return formatted_text
    except json.JSONDecodeError:
        print(f"Error decoding JSON in format_chunks_for_llm: {traceback.format_exc()}")
        return "Error: Could not parse the retrieved information snippets for formatting.\n"
    except Exception as e:
        print(f"An unexpected error occurred in format_chunks_for_llm: {e}\n{traceback.format_exc()}")
        return f"An unexpected error occurred while formatting snippets: {str(e)}\n"

# --- PROMPT FUNCTION: Craft instructions for final answer + citation links ---
YOUR_APP_DOMAIN = "www.stackifier.com" # As per your example, make this configurable if needed

def create_final_answer_instructions(user_original_query: str, formatted_snippets_text: str) -> str:
    """
    Creates the detailed instructions for the LLM to generate the final answer,
    incorporating the formatted snippets and specifying the Markdown citation format.
    """
    instructions = f"""
Based on the user's original question: "{user_original_query}"
And the following information snippets I retrieved for you:

{formatted_snippets_text}

Please perform the following steps:
1. Carefully review the information snippets.
2. Answer the user's original question using ONLY the information present in these snippets. Do not use any prior knowledge or external information.
3. If the answer cannot be found in the provided snippets, clearly state that the information is not available in the documents. Your response should still be polite and acknowledge the query.
4. After your answer, include a "Sources:" section on a new line.
5. In the "Sources:" section, list each snippet number you explicitly used to construct your answer, along with its corresponding document name and section ID, formatted as a Markdown link.
6. The Markdown link format MUST be exactly: `[<Snippet Number>. (<Source Document Name>)](https://{YOUR_APP_DOMAIN}/document?section_id=<Source Section ID>)`
   - Replace `<Snippet Number>` with the number from the "[Snippet X]" heading in the snippets I provided (e.g., 1, 2).
   - Replace `<Source Document Name>` with the 'Source Document' name provided for that snippet.
   - Replace `<Source Section ID>` with the 'Source Section ID' provided for that snippet.

Example of a "Sources:" section entry if Snippet 1 was from 'report.pdf' with section ID 'abc-123':
Sources:
[1. (report.pdf)](https://{YOUR_APP_DOMAIN}/document?section_id=abc-123)

If you use information from multiple snippets, list them all under the "Sources:" heading, each on a new line using the specified Markdown link format.
If no snippets were found or if the snippets do not contain the answer, do not include a "Sources:" section, but state clearly that the information could not be found in the provided documents based on the provided documents.
"""
    return instructions


def retrieve_financial_chunks(
    query_text: str,
    user_id: str,
    match_count: int = 5,
    doc_specific_type: str = None,
    company_name: str = None,
    doc_year_start: int = None,
    doc_year_end: int = None,
    doc_quarter: int = None
) -> str:
    """
    Retrieves relevant financial document chunks from Supabase based on a query
    and optional metadata filters for a specific user.

    Returns:
        A JSON string representation of the list of retrieved chunk dictionaries,
        or a JSON string with an error message.
    IMPORTANT: Ensure the RPC 'match_chunks' returns 'document_filename' (from documents table)
               and 'section_id' (the section UUID from sections table) for each chunk,
               in addition to 'chunk_text', 'section_heading', etc.
    """
    print(f"\n--- Executing Tool: retrieve_financial_chunks ---")
    print(f"  Query: '{query_text}'")
    print(f"  User ID: {user_id}")
    print(f"  Filters: Type={doc_specific_type}, Company={company_name}, Year={doc_year_start}-{doc_year_end}, Qtr={doc_quarter}")

    global openai_client, supabase_service # Ensure these are properly initialized in the global scope
    if openai_client is None or supabase_service is None:
         return json.dumps({"error": "Clients not initialized. OpenAI or Supabase service is None."})

    try:
        # --- Step 1: Generate Embedding for the query ---
        query_embedding_list = openai_client.get_embeddings([query_text])
        if not query_embedding_list:
            print(f"  Error: Failed to generate query embedding.")
            return json.dumps({"error": "Failed to generate query embedding."})
        query_embedding = query_embedding_list[0]
        print(f"  Query embedding generated.")

        # --- Step 2: Call Supabase RPC to find matching chunks ---
        print(f"  Calling Supabase RPC 'match_chunks'...")
        # Ensure your 'match_chunks' RPC in Supabase handles the filters and returns the required fields.
        response = supabase_service.client.rpc(
            'match_chunks', # This RPC must exist and return document_filename and section_id
            {
                'query_embedding': query_embedding,
                'match_count': match_count,
                'user_id': user_id,
                'p_doc_specific_type': doc_specific_type,
                'p_company_name': company_name,
                'p_doc_year_start': doc_year_start,
                'p_doc_year_end': doc_year_end,
                'p_doc_quarter': doc_quarter
            }
        ).execute()

        # --- Step 3: Process RPC response ---
        if response.data is not None: # Check if data is not None (even empty list is valid data)
            print(f"  Retrieved {len(response.data)} chunks from Supabase.")
            processed_data = []
            for chunk_dict in response.data: # chunk_dict is a dictionary from the RPC
                 processed_chunk = {}
                 for key, value in chunk_dict.items():
                     # Convert UUIDs to strings for JSON serialization if needed
                     if isinstance(value, uuid.UUID):
                         processed_chunk[key] = str(value)
                     else:
                         processed_chunk[key] = value

                 # --- Critical Check: Ensure required keys for citation are present ---
                 # The RPC *must* return 'document_filename' and 'section_id'.
                 # Add placeholders/warnings if they are missing, but the RPC should be fixed.
                 if 'document_filename' not in processed_chunk or processed_chunk['document_filename'] is None:
                     print(f"  Warning: RPC 'match_chunks' did not return 'document_filename' for a chunk.")
                     processed_chunk['document_filename'] = 'RPC_Missing_Doc_Name'
                 if 'section_id' not in processed_chunk or processed_chunk['section_id'] is None:
                     print(f"  Warning: RPC 'match_chunks' did not return 'section_id' for a chunk.")
                     # Fallback to chunk id string if section id is missing, though not ideal for section link
                     processed_chunk['section_id'] = str(processed_chunk.get('id', 'RPC_Missing_Section_ID'))
                 # Also ensure chunk_text and section_heading are present for formatting
                 if 'chunk_text' not in processed_chunk:
                      processed_chunk['chunk_text'] = 'Chunk text missing from RPC.'
                 if 'section_heading' not in processed_chunk:
                      processed_chunk['section_heading'] = 'Section heading missing from RPC.'


                 processed_data.append(processed_chunk)

            result_json_string = json.dumps(processed_data, indent=2)
            # Avoid printing potentially massive JSON result entirely
            print(f"  Returning JSON result ({len(result_json_string)} chars, first 500 for brevity):\n{result_json_string[:500]}...")
            return result_json_string
        elif hasattr(response, 'error') and response.error:
             error_msg = f"Supabase RPC 'match_chunks' error: {response.error.message if hasattr(response.error, 'message') else response.error}"
             print(f"  Error: {error_msg}")
             return json.dumps({"error": error_msg})
        else:
            # This case handles unexpected response structures where data and error are missing
            print("  Received unexpected response structure from Supabase RPC 'match_chunks'.")
            print(f"  Response object type: {type(response)}")
            # Attempt to get any response content if available
            try:
                print(f"  Response data: {response.data}")
                print(f"  Response error: {response.error}")
            except Exception as print_exc:
                print(f"  Could not print response details: {print_exc}")
            return json.dumps({"error": "Unexpected response from Supabase RPC. Data and error fields were not accessible or were None when expected."})


    except Exception as e:
        # Catch any other exceptions during embedding or RPC call setup/execution
        print(f"An unexpected error occurred during chunk retrieval: {str(e)}\n{traceback.format_exc()}")
        return json.dumps({"error": f"An unexpected error occurred during chunk retrieval: {str(e)}"})


# --- Define Function Declaration for Gemini ---
# This tells the LLM about the tool it can use
retrieve_chunks_declaration = {
    "name": "retrieve_financial_chunks",
    "description": "Searches and retrieves relevant text chunks from the user's uploaded financial documents based on their query and optional filters like company name, document type, year range, or quarter. Always use this tool to find information before answering questions about the user's financial documents.",
    "parameters": {
        "type": "object",
        "properties": {
            "query_text": {
                "type": "string",
                "description": "The user's original question or a refined search query based on their question.",
            },
            "match_count": {
                "type": "integer",
                "description": "The maximum number of relevant text chunks to return. Default is 5. Use a reasonable number like 5-10.",
            },
            "doc_specific_type": {
                "type": "string",
                # Ensure FinancialDocSpecificType is correctly imported and has a .value attribute
                "description": f"Filter results to a specific document type. Examples: {', '.join([item.value for item in FinancialDocSpecificType if item != FinancialDocSpecificType.UNKNOWN and item.value is not None])}. Leave empty if no specific type is mentioned.",
            },
            "company_name": {
                "type": "string",
                "description": "Filter results to a specific company name mentioned in the query. Use the most likely name if variations exist (e.g., 'Tesla' for 'Tesla, Inc.'). Leave empty if no company is mentioned.",
            },
            "doc_year_start": {
                "type": "integer",
                "description": "The starting fiscal year for filtering (e.g., 2021). Extract from the user's query if a year or date range is specified. Leave empty if no start year is specified.",
            },
            "doc_year_end": {
                "type": "integer",
                "description": "The ending fiscal year for filtering (e.g., 2021). Use the same year as start year if only one year is mentioned. Leave empty if no end year is specified.",
            },
            "doc_quarter": {
                "type": "integer",
                "description": "Filter results to a specific fiscal quarter (1, 2, 3, or 4). Extract if mentioned in the query. Use -1 or leave empty if no quarter is mentioned.",
            },
        },
        "required": ["query_text"] # query_text is mandatory for the tool
    },
}


# --- Configuration and Globals ---
load_dotenv()
TEST_EMAIL = os.environ.get("TEST_EMAIL")
TEST_PASSWORD = os.environ.get("TEST_PASSWORD")
if not TEST_EMAIL or not TEST_PASSWORD:
    raise ValueError("TEST_EMAIL and TEST_PASSWORD must be set in your .env file.")

# Keep track of conversation history. This is crucial for multi-turn interactions.
# Gemini API expects history in a specific turn-based format (user, model, user, model, ...)
conversation_history = []

# --- User Query ---
user_query = "Whats the Gross Carrying Amount for Total intangible assets for tesla in 2021? Create a report of tesla for 2021 in markdown i can copy."
print(f"\n--- User Query ---")
print(user_query)

# --- Initialize Clients and Authenticate with Supabase ---
openai_client = None
gemini_client = None
supabase_service = None
authenticated_user_id_str = None
auth_client = None

try:
    print("\n--- Initializing clients and authenticating ---")
    # Initialize OpenAI client for embeddings
    openai_client = OpenAIClient()
    # Initialize Gemini client for LLM interactions
    # GeminiClient assumes GEMINI_API_KEY is in .env
    gemini_client = GeminiClient()

    # Initialize Supabase client and authenticate test user
    supabase_url = os.environ.get("SUPABASE_URL")
    # Use ANON_KEY for RLS-enabled access
    supabase_key = os.environ.get("SUPABASE_ANON_KEY")
    if not supabase_url or not supabase_key:
        raise ValueError("SUPABASE_URL and SUPABASE_ANON_KEY must be set in your .env file.")

    auth_client: Client = create_client(supabase_url, supabase_key)
    print("Supabase client created.")

    # Authenticate the test user
    print(f"Attempting to sign in with email: {TEST_EMAIL}")
    auth_response = auth_client.auth.sign_in_with_password(
        {"email": TEST_EMAIL, "password": TEST_PASSWORD}
    )

    # Check if authentication was successful and user object is available
    if not auth_response or not auth_response.user:
        error_detail = auth_response.error.message if hasattr(auth_response, 'error') and auth_response.error else "Unknown authentication error"
        raise ConnectionError(f"Supabase authentication failed: {error_detail}. Check credentials and Supabase Auth settings.")

    # Extract the authenticated user ID (UUID)
    authenticated_user_id_str = str(auth_response.user.id)
    print(f"Authentication successful. User ID: {authenticated_user_id_str}")

    # Initialize SupabaseService with the authenticated client
    supabase_service = SupabaseService(supabase_client=auth_client)
    print("Clients initialized and authenticated.")

except Exception as e:
    print(f"Initialization or Authentication Error: {str(e)}\n{traceback.format_exc()}")
    sys.exit(1) # Exit if initialization fails

# --- Define the Tool for Gemini ---
# This tells the LLM about the function it has access to
retrieval_tool = types.Tool(function_declarations=[retrieve_chunks_declaration])
# Using the model from your log, or a recommended latest flash version
# gemini_model_name = "gemini-2.5-flash-preview-04-17" # If you must use this specific preview
gemini_model_name = "gemini-2.5-flash-preview-04-17" # Using latest flash model recommended for general use


# --- Function Calling Loop ---
# This block simulates the interaction between the LLM and your tool
try:
    # 1. First call to LLM: Send user query and retrieval tool definition
    print(f"\n--- Sending initial query to Gemini ({gemini_model_name}) ---")
    # Add the user's initial query to the conversation history
    conversation_history.append(types.Content(role="user", parts=[types.Part(text=user_query)]))

    # Print history for debugging using the serialization helper
    print(f"  Conversation History before first call:\n{json.dumps(serialize_conversation_history(conversation_history), indent=2)}")


    # Send the conversation history and available tools to Gemini
    # FIX: Use 'config' parameter with types.GenerateContentConfig for tools
    response = gemini_client.client.models.generate_content( # Correctly using gemini_client.client
        model=gemini_model_name,
        contents=conversation_history,
        config=types.GenerateContentConfig(tools=[retrieval_tool]) # Pass tools via config with GenerateContentConfig
    )

    print(f"\n--- Received response from first Gemini call ---")
    # Print the full response structure for inspection (can be verbose)
    # print(response)

    # Check if the response contains candidates and parts
    if not response.candidates or not response.candidates[0].content or not response.candidates[0].content.parts:
        print("Error: Unexpected response structure or no candidates/parts from Gemini's first call.")
        print(f"Full response object: {response}") # Print the full response object for debugging
        # Check if there's a prompt feedback error
        if hasattr(response, 'prompt_feedback') and response.prompt_feedback:
            print(f"Prompt Feedback: {response.prompt_feedback}")
        sys.exit(1) # Exit if initial call fails

    # Get the first part of the model's response
    model_response_content = response.candidates[0].content
    message_part = model_response_content.parts[0]

    # Add the model's response (which might contain a function call) to history
    conversation_history.append(model_response_content)
    print(f"\n  Model's response content (first call) added to history.")
    # Print model response content for debugging using the serialization helper
    # print(f"  Model response content:\n{serialize_conversation_history([model_response_content])[0]}")


    # 2. Check if LLM requested a function call
    if hasattr(message_part, 'function_call') and message_part.function_call:
        function_call = message_part.function_call
        print(f"\n--- Gemini requested function call: '{function_call.name}' ---")
        # LLM args is a Pydantic model-like object, convert to dict for easier use
        tool_args = dict(function_call.args)
        print(f"  Raw Arguments from LLM: {tool_args}")

        # 3. Execute the function if it's the one we defined
        if function_call.name == "retrieve_financial_chunks":
            print("  Recognized 'retrieve_financial_chunks' call.")
            print(f"  Extracted tool_args before adding user_id: {tool_args}")

            # --- IMPORTANT: Add authenticated user_id to tool arguments ---
            # The user_id comes from authentication, not from the LLM's interpretation of the query.
            # It must be passed to our internal function for RLS enforcement.
            tool_args['user_id'] = authenticated_user_id_str
            print(f"  Tool_args *including* user_id for execution: {tool_args}")

            # Call the actual Python function that interacts with Supabase/OpenAI
            function_result_json = retrieve_financial_chunks(**tool_args)
            print(f"\n--- Finished executing retrieve_financial_chunks ---")
            # Print a summary of the function result (the JSON string)
            print(f"  Function result (JSON string, first 500 chars):\n{function_result_json[:500]}...")


            # 4. Second call to LLM: Send function result back AND provide instructions for final answer + citations
            print("\n--- Preparing enriched context and instructions for final Gemini call ---")

            # A. Add the raw function response (as structured data) to the conversation history
            # This allows the LLM to "see" the specific data returned by the tool.
            # The response must be a dictionary matching the expected tool response structure.
            try:
                function_response_data = json.loads(function_result_json)
            except json.JSONDecodeError:
                 print(f"Error decoding function result JSON: {function_result_json[:200]}...")
                 function_response_data = {"error": "Invalid JSON from tool."} # Send error back to LLM

            function_response_part = types.Part.from_function_response(
                name=function_call.name,
                response={"result": function_response_data} # Wrap the result in a 'result' key as used in docs
            )
            # Per Gemini docs, the function response should be in a "user" role part that follows
            # the "model" role part containing the function_call.
            # Our conversation_history structure now:
            # 1. User: Original query
            # 2. Model: Function call (`model_response_content`)
            # 3. User: Function response (`function_response_part`)
            conversation_history.append(
                 types.Content(role="user", parts=[function_response_part]) # role="user" for function response
            )
            print(f"  Raw function response part added to history.")
            # Print for debugging
            # print(f"  Function response part:\n{serialize_conversation_history([types.Content(role='user', parts=[function_response_part])])[0]}")


            # B. Format the retrieved chunks into a readable text block for the LLM
            # This is the context the LLM will actually read to answer the question.
            formatted_snippets_text = format_chunks_for_llm(function_result_json)

            # C. Create the new detailed instructions for answer generation and citation formatting
            final_instructions_text = create_final_answer_instructions(user_query, formatted_snippets_text)

            # D. Add these new instructions as another user message to guide the LLM's final response
            # This user turn contains the human-readable snippets and the citation instructions.
            conversation_history.append(
                types.Content(role="user", parts=[types.Part(text=final_instructions_text)])
            )
            print(f"  Formatted snippets and citation instructions added to history.")
            # Print instructions for debugging
            # print(f"  Final instructions text (first 500 chars for brevity):\n{final_instructions_text[:500]}...")

            # Print the conversation history *before* the final call for debugging
            print(f"\n  Conversation History before second call (final answer generation):\n{json.dumps(serialize_conversation_history(conversation_history), indent=2)}")

            # --- Step 5: Generate the final response ---
            # Send the complete conversation history (including tool call and response, and formatted context)
            # FIX: Remove 'config' parameter in the second call as tools are likely not needed here
            final_response = gemini_client.client.models.generate_content(
                model=gemini_model_name,
                contents=conversation_history
                # Tools are generally not needed for the final answer generation turn.
                # If the LLM needs to call a tool *again* based on the function result,
                # you would re-enable the tools here using:
                # config=types.GenerateContentConfig(tools=[retrieval_tool])
            )

            print(f"\n--- Received response from second Gemini call (Final Answer) ---")
            # Print the full final response object for inspection
            # print(final_response)

            # --- Step 6: Extract and print the final answer ---
            if final_response.candidates and final_response.candidates[0].content and final_response.candidates[0].content.parts:
                 final_model_response_content = final_response.candidates[0].content
                 # Add the final model response to history if you plan more turns
                 conversation_history.append(final_model_response_content)
                 # Print for debugging
                 # print(f"\n  Final model's response content added to history: {serialize_conversation_history([final_model_response_content])[0]}")

                 # The final answer text should be in the text part
                 print("\n\n****************************************")
                 print("--- Final Answer Text from Gemini ---")
                 print("****************************************")
                 print(final_response.text) # This should now include Markdown citation links
                 print("****************************************\n")
            else:
                 print("Error: No final response text found after sending function result.")
                 print(f"Full final response object: {final_response}")
                 if hasattr(final_response, 'prompt_feedback') and final_response.prompt_feedback:
                     print(f"Final Response Prompt Feedback: {final_response.prompt_feedback}")


        else:
            # This branch executes if the LLM requested a function call, but it wasn't
            # the 'retrieve_financial_chunks' function we defined.
            print(f"Warning: LLM requested unknown function '{function_call.name}'")
            # In a real application, you might send this back to the LLM as a "tool error"
            # or inform the user. For a simple notebook, we just report and stop.
            print("Stopping execution due to unknown function call.")

    else:
        # This branch executes if the LLM decided to answer the user's query directly
        # without requesting a function call. This might happen if the query is simple
        # or if the LLM determines it doesn't need the tool.
        print("\n--- Gemini decided to answer directly (No Function Call Requested) ---")
        if hasattr(message_part, 'text') and message_part.text is not None:
            print(message_part.text)
            # Add the direct model response to history
            conversation_history.append(model_response_content)
        else:
            print("No text response found in the initial call and no function call made.")
            # Print message part for debugging
            # print(f"Message part was: {message_part}")
        print("Stopping execution after direct answer or unexpected initial response.")

except Exception as e:
    print(f"\nAn unexpected error occurred during the Gemini interaction: {str(e)}\n{traceback.format_exc()}")


--- User Query ---
Whats the Gross Carrying Amount for Total intangible assets for tesla in 2021? Create a report of tesla for 2021 in markdown i can copy.

--- Initializing clients and authenticating ---
Initialized OpenAI client with model: text-embedding-3-small
Initializing Gemini client with API key: AIz...yQ
Supabase client created.
Attempting to sign in with email: wbryanlai@gmail.com
Authentication successful. User ID: e222921f-cfdc-4a05-8cf2-aea13004bcf2
SupabaseService initialized with provided client.
Clients initialized and authenticated.

--- Sending initial query to Gemini (gemini-2.5-flash-preview-04-17) ---
  Conversation History before first call:
[
  {
    "role": "user",
    "parts": [
      {
        "text": "Whats the Gross Carrying Amount for Total intangible assets for tesla in 2021? Create a report of tesla for 2021 in markdown i can copy."
      }
    ]
  }
]

--- Received response from first Gemini call ---

  Model's response content (first call) added to hi